In [151]:
import requests
import pandas as pd
from tqdm import tqdm

In [152]:
url_uf = "https://resultados.tse.jus.br/oficial/ele2022/544/config/mun-e000544-cm.json"
requisicao_uf = requests.get(url_uf)
requisicao_uf = requisicao_uf.json()

In [163]:
from joblib import Parallel, delayed

# função para rodar código em paralelo

def pegar_dados_api(uf):
    lista_candidatos = []

    sigla_uf = uf["cd"]
    estado = uf["ds"]

    #if sigla_uf == "AL":


    for municipio in uf["mu"]:

        nome_mun = municipio["nm"]
        cod_tse = municipio["cd"]
        cod_ibge = municipio["cdi"]

        url_mun = f"https://resultados.tse.jus.br/oficial/ele2022/544/dados/{sigla_uf.lower()}/{sigla_uf.lower()}{cod_tse}-c0001-e000544-v.json"
        requisicao_mun = requests.get(url_mun)
        requisicao_mun = requisicao_mun.json()

        for candidato in requisicao_mun["abr"][0]["cand"]:

            lista_candidatos.append({
                "uf": sigla_uf,
                "estado": estado,
                "nome_mun": nome_mun,
                "cod_tse": cod_tse,
                "cod_ibge": cod_ibge,
                "posicao_canditato": candidato["seq"],
                "num_canditato": candidato["n"],
                "situacao": candidato["st"],
                "votos_totais": candidato["vap"],
                "perc_votos_totais": candidato["pvap"],
            })

    return lista_candidatos


In [164]:
# vídeo explicativo de como funciona a biblioteca para rodar código em paralelo https://www.youtube.com/watch?v=maj47xd_R7g&t=906

resultado = Parallel(n_jobs=-1)(delayed(pegar_dados_api)(uf) for uf in requisicao_uf["abr"])
#print(resultado)

In [170]:
# organizando os dados em apenas uma lista
lista_dados_coletados = []

for uf in resultado:
    for municipio in uf:
        lista_dados_coletados.append(municipio)

In [176]:
df_dados = pd.DataFrame(lista_dados_coletados)

# transportando dados de linhas para colunas com a função "pivote"
dados_agredados = df_dados.pivot(index=["cod_tse", "cod_ibge", "nome_mun", "uf", "estado"],
                                columns='num_canditato', 
                                values="perc_votos_totais")
dados_agredados.head()

,,,,num_canditato,12,13,14,15,16,21,22,27,30,44,80
cod_tse,cod_ibge,nome_mun,uf,estado,,,,,,,,,,,
00019,1100106,GUAJARÁ-MIRIM,RO,RONDÔNIA,"2,06","34,19","0,05","3,82","0,00","0,00","59,14","0,01","0,15","0,53","0,03"
00035,1100205,PORTO VELHO,RO,RONDÔNIA,"2,94","34,21","0,11","4,55","0,02","0,03","56,83","0,02","0,42","0,81","0,04"
00051,1100122,JI-PARANÁ,RO,RONDÔNIA,"1,92","24,24","0,08","3,08","0,02","0,03","69,79","0,01","0,26","0,55","0,02"
00078,1100023,ARIQUEMES,RO,RONDÔNIA,"1,74","20,97","0,06","2,85","0,01","0,03","73,52","0,00","0,29","0,48","0,03"
00094,1100049,CACOAL,RO,RONDÔNIA,"2,10","24,04","0,08","3,28","0,01","0,02","69,55","0,01","0,34","0,55","0,03"


In [177]:
# reorganizando os nomes das colunas
dados_agredados = dados_agredados.reset_index()
dados_agredados.head()

num_canditato,cod_tse,cod_ibge,nome_mun,uf,estado,12,13,14,15,16,21,22,27,30,44,80
0,00019,1100106,GUAJARÁ-MIRIM,RO,RONDÔNIA,"2,06","34,19","0,05","3,82","0,00","0,00","59,14","0,01","0,15","0,53","0,03"
1,00035,1100205,PORTO VELHO,RO,RONDÔNIA,"2,94","34,21","0,11","4,55","0,02","0,03","56,83","0,02","0,42","0,81","0,04"
2,00051,1100122,JI-PARANÁ,RO,RONDÔNIA,"1,92","24,24","0,08","3,08","0,02","0,03","69,79","0,01","0,26","0,55","0,02"
3,00078,1100023,ARIQUEMES,RO,RONDÔNIA,"1,74","20,97","0,06","2,85","0,01","0,03","73,52","0,00","0,29","0,48","0,03"
4,00094,1100049,CACOAL,RO,RONDÔNIA,"2,10","24,04","0,08","3,28","0,01","0,02","69,55","0,01","0,34","0,55","0,03"


In [179]:
# dar nota de 1 a 4 (a depender do resultado) para ajudar na visualização no datawrapper

dados_status = []

for row in dados_agredados.values:
    
    lula = float(row[6].replace(',', '.'))
    bolsonaro = float(row[11].replace(',', '.'))

    if lula < 50 and lula > bolsonaro:
        status = 2 # Lula ganha mas tem menos que 50%    

    if lula >= 50:
        status = 1 # Lula ganha com mais de 50%

    if bolsonaro < 50 and bolsonaro > lula:
        status = 3 # Lula ganha mas tem menos que 50%    

    if bolsonaro >= 50:
        status = 4 # Lula ganha com mais de 50%

    dados_status.append(status)
    

In [180]:
# adicionando dados em uma nova coluna do dataframe
dados_agredados['status'] = dados_status

In [182]:
df_dados.to_csv("dados/votos_presidente_municipios_2022.csv")

#### ---- FIM ----

### execucao do código normal
(sem dar nota de status)

In [30]:
lista_candidatos = [] 

for uf in tqdm(requisicao_uf["abr"]):
    
    #print(uf["ds"])
    if sigla_uf == "AL":
        sigla_uf = uf["cd"]
        estado = uf["ds"]

        #print("------")
        for municipio in uf["mu"]:

            nome_mun = municipio["nm"]
            cod_tse = municipio["cd"]
            cod_ibge = municipio["cdi"]

            url_mun = f"https://resultados.tse.jus.br/oficial/ele2022/544/dados/{sigla_uf.lower()}/{sigla_uf.lower()}{cod_tse}-c0001-e000544-v.json"
            requisicao_mun = requests.get(url_mun)
            requisicao_mun = requisicao_mun.json()

            for candidato in requisicao_mun["abr"][0]["cand"]:

                lista_candidatos.append({
                    "uf": sigla_uf,
                    "estado": estado,
                    "nome_mun": nome_mun,
                    "cod_tse": cod_tse,
                    "cod_ibge": cod_ibge,
                    "posicao_canditato": candidato["seq"],
                    "num_canditato": candidato["n"],
                    "situacao": candidato["st"],
                    "votos_totais": candidato["vap"],
                    "perc_votos_totais": candidato["pvap"],
                })


100%|██████████| 28/28 [22:36<00:00, 48.46s/it]


In [61]:
print(len(lista_candidatos))

63261


In [32]:
df_dados = pd.DataFrame(lista_candidatos)
df_dados.to_csv("dados/votos_presidente_municipios_2022.csv")